# Transformations on a JSON file using Pandas

This is a notebook for the medium article [Transformations on a JSON file using Pandas](https://medium.com/@nachovargas/transformations-on-a-json-file-using-pandas-eba831181a96)

Please check out article for instructions

License: [MIT](https://opensource.org/licenses/MIT)

In [11]:
import pandas as pd
import requests
import json

## 1. Explode

In [2]:
data = {
    'col_1': [True, False],
    'col_2': [['one', 'two'], ['three']]
}
df = pd.DataFrame(data)
df

,col_1,col_2
0,True,"[one, two]"
1,False,[three]


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col_1   2 non-null      bool  
 1   col_2   2 non-null      object
dtypes: bool(1), object(1)
memory usage: 146.0+ bytes


In [4]:
exploded_df = df.explode('col_2')
exploded_df

,col_1,col_2
0,True,one
0,True,two
1,False,three


In [5]:
exploded_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3 entries, 0 to 1
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   col_1   3 non-null      bool  
 1   col_2   3 non-null      object
dtypes: bool(1), object(1)
memory usage: 51.0+ bytes


## 2. JSON Normalize

In [6]:
data = [
  {
    "id": 1,
    "name": {
      "first": "Coleen",
      "last": "Volk"
    }
  },
  {
    "name": {
      "given": "Mark",
      "family": "Regner"
    }
  },
  {
    "id": 2,
    "name": "Faye Raker"
  },
  
]
df = pd.json_normalize(data)
df

,id,name.first,name.last,name.given,name.family,name
0,1.0,Coleen,Volk,NaN,NaN,NaN
1,NaN,NaN,NaN,Mark,Regner,NaN
2,2.0,NaN,NaN,NaN,NaN,Faye Raker


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           2 non-null      float64
 1   name.first   1 non-null      object 
 2   name.last    1 non-null      object 
 3   name.given   1 non-null      object 
 4   name.family  1 non-null      object 
 5   name         1 non-null      object 
dtypes: float64(1), object(5)
memory usage: 272.0+ bytes


In [8]:
df = pd.json_normalize(data, sep=' | ')
df

,id,name | first,name | last,name | given,name | family,name
0,1.0,Coleen,Volk,NaN,NaN,NaN
1,NaN,NaN,NaN,Mark,Regner,NaN
2,2.0,NaN,NaN,NaN,NaN,Faye Raker


In [9]:
df = pd.json_normalize(data, max_level=0)
df

,id,name
0,1.0,"{'first': 'Coleen', 'last': 'Volk'}"
1,NaN,"{'given': 'Mark', 'family': 'Regner'}"
2,2.0,Faye Raker


In [10]:
df = pd.json_normalize(data, max_level=1)
df

,id,name.first,name.last,name.given,name.family,name
0,1.0,Coleen,Volk,NaN,NaN,NaN
1,NaN,NaN,NaN,Mark,Regner,NaN
2,2.0,NaN,NaN,NaN,NaN,Faye Raker


## 3. Example

We will get the example products from the dummyjson website.

In [21]:

raw_data = requests.get('https://dummyjson.com/carts')
data = raw_data.json().get('carts')
data

[{'id': 1,
  'products': [{'id': 59,
    'title': 'Spring and summershoes',
    'price': 20,
    'quantity': 3,
    'total': 60,
    'discountPercentage': 8.71,
    'discountedPrice': 55},
   {'id': 88,
    'title': 'TC Reusable Silicone Magic Washing Gloves',
    'price': 29,
    'quantity': 2,
    'total': 58,
    'discountPercentage': 3.19,
    'discountedPrice': 56},
   {'id': 18,
    'title': 'Oil Free Moisturizer 100ml',
    'price': 40,
    'quantity': 2,
    'total': 80,
    'discountPercentage': 13.1,
    'discountedPrice': 70},
   {'id': 95,
    'title': 'Wholesale cargo lashing Belt',
    'price': 930,
    'quantity': 1,
    'total': 930,
    'discountPercentage': 17.67,
    'discountedPrice': 766},
   {'id': 39,
    'title': 'Women Sweaters Wool',
    'price': 600,
    'quantity': 2,
    'total': 1200,
    'discountPercentage': 17.2,
    'discountedPrice': 994}],
  'total': 2328,
  'discountedTotal': 1941,
  'userId': 97,
  'totalProducts': 5,
  'totalQuantity': 10},
 {'id'

In [22]:
df = pd.DataFrame.from_dict(data)
df

,id,products,total,discountedTotal,userId,totalProducts,totalQuantity
0,1,"[{'id': 59, 'title': 'Spring and summershoes',...",2328,1941,97,5,10
1,2,"[{'id': 96, 'title': 'lighting ceiling kitchen...",3023,2625,30,5,10
2,3,"[{'id': 37, 'title': 'ank Tops for Womens/Girl...",460,403,63,5,10
3,4,"[{'id': 36, 'title': 'Sleeve Shirt Womens', 'p...",553,493,83,5,10
4,5,"[{'id': 23, 'title': 'Orange Essence Food Flav...",844,745,58,5,10
5,6,"[{'id': 53, 'title': 'printed high quality T s...",1454,1276,26,5,12
6,7,"[{'id': 61, 'title': 'Leather Straps Wristwatc...",588,519,56,5,10
7,8,"[{'id': 45, 'title': 'Malai Maxi Dress', 'pric...",1129,952,1,5,9
8,9,"[{'id': 74, 'title': 'Leather Hand Bag', 'pric...",3608,3371,91,5,10
9,10,"[{'id': 75, 'title': 'Seven Pocket Women Bag',...",9064,8205,13,5,9


### 3.1 Explode the example

In [30]:
exploded_df = df.explode('products')
exploded_df

,id,products,total,discountedTotal,userId,totalProducts,totalQuantity
0,1,"{'id': 59, 'title': 'Spring and summershoes', ...",2328,1941,97,5,10
0,1,"{'id': 88, 'title': 'TC Reusable Silicone Magi...",2328,1941,97,5,10
0,1,"{'id': 18, 'title': 'Oil Free Moisturizer 100m...",2328,1941,97,5,10
0,1,"{'id': 95, 'title': 'Wholesale cargo lashing B...",2328,1941,97,5,10
0,1,"{'id': 39, 'title': 'Women Sweaters Wool', 'pr...",2328,1941,97,5,10
...,...,...,...,...,...,...,...
19,20,"{'id': 66, 'title': 'Steel Analog Couple Watch...",315,279,75,5,8
19,20,"{'id': 59, 'title': 'Spring and summershoes', ...",315,279,75,5,8
19,20,"{'id': 29, 'title': 'Handcraft Chinese style',...",315,279,75,5,8
19,20,"{'id': 32, 'title': 'Sofa for Coffe Cafe', 'pr...",315,279,75,5,8


In [31]:
exploded_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 19
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               100 non-null    int64 
 1   products         100 non-null    object
 2   total            100 non-null    int64 
 3   discountedTotal  100 non-null    int64 
 4   userId           100 non-null    int64 
 5   totalProducts    100 non-null    int64 
 6   totalQuantity    100 non-null    int64 
dtypes: int64(6), object(1)
memory usage: 6.2+ KB


### 3.2 Normalize the example

In [32]:
normalized_df = pd.json_normalize(exploded_df['products'])
normalized_df

,id,title,price,quantity,total,discountPercentage,discountedPrice
0,59,Spring and summershoes,20,3,60,8.71,55
1,88,TC Reusable Silicone Magic Washing Gloves,29,2,58,3.19,56
2,18,Oil Free Moisturizer 100ml,40,2,80,13.10,70
3,95,Wholesale cargo lashing Belt,930,1,930,17.67,766
4,39,Women Sweaters Wool,600,2,1200,17.20,994
...,...,...,...,...,...,...,...
95,66,Steel Analog Couple Watches,35,3,105,3.23,102
96,59,Spring and summershoes,20,1,20,8.71,18
97,29,Handcraft Chinese style,60,1,60,15.34,51
98,32,Sofa for Coffe Cafe,50,1,50,15.59,42


In [33]:
normalized_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  100 non-null    int64  
 1   title               100 non-null    object 
 2   price               100 non-null    int64  
 3   quantity            100 non-null    int64  
 4   total               100 non-null    int64  
 5   discountPercentage  100 non-null    float64
 6   discountedPrice     100 non-null    int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 5.6+ KB


### 3.3 Join

In [35]:
exploded_df.drop(['products'], axis=1, inplace=True)
exploded_df.reset_index(inplace=True)
exploded_df

,index,id,total,discountedTotal,userId,totalProducts,totalQuantity
0,0,1,2328,1941,97,5,10
1,0,1,2328,1941,97,5,10
2,0,1,2328,1941,97,5,10
3,0,1,2328,1941,97,5,10
4,0,1,2328,1941,97,5,10
...,...,...,...,...,...,...,...
95,19,20,315,279,75,5,8
96,19,20,315,279,75,5,8
97,19,20,315,279,75,5,8
98,19,20,315,279,75,5,8


In [36]:
joined_df = exploded_df.join(normalized_df, lsuffix='_cart', rsuffix='_product')
joined_df

,index,id_cart,total_cart,discountedTotal,userId,totalProducts,totalQuantity,id_product,title,price,quantity,total_product,discountPercentage,discountedPrice
0,0,1,2328,1941,97,5,10,59,Spring and summershoes,20,3,60,8.71,55
1,0,1,2328,1941,97,5,10,88,TC Reusable Silicone Magic Washing Gloves,29,2,58,3.19,56
2,0,1,2328,1941,97,5,10,18,Oil Free Moisturizer 100ml,40,2,80,13.10,70
3,0,1,2328,1941,97,5,10,95,Wholesale cargo lashing Belt,930,1,930,17.67,766
4,0,1,2328,1941,97,5,10,39,Women Sweaters Wool,600,2,1200,17.20,994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,19,20,315,279,75,5,8,66,Steel Analog Couple Watches,35,3,105,3.23,102
96,19,20,315,279,75,5,8,59,Spring and summershoes,20,1,20,8.71,18
97,19,20,315,279,75,5,8,29,Handcraft Chinese style,60,1,60,15.34,51
98,19,20,315,279,75,5,8,32,Sofa for Coffe Cafe,50,1,50,15.59,42


In [37]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               100 non-null    int64  
 1   id_cart             100 non-null    int64  
 2   total_cart          100 non-null    int64  
 3   discountedTotal     100 non-null    int64  
 4   userId              100 non-null    int64  
 5   totalProducts       100 non-null    int64  
 6   totalQuantity       100 non-null    int64  
 7   id_product          100 non-null    int64  
 8   title               100 non-null    object 
 9   price               100 non-null    int64  
 10  quantity            100 non-null    int64  
 11  total_product       100 non-null    int64  
 12  discountPercentage  100 non-null    float64
 13  discountedPrice     100 non-null    int64  
dtypes: float64(1), int64(12), object(1)
memory usage: 11.1+ KB


### 3.4 Transformations

In [38]:
threshold = 100

big_sale_col = joined_df.apply(lambda row: row.total_product >= threshold, axis=1)
big_sale_col

0     False
1     False
2     False
3      True
4      True
      ...  
95     True
96    False
97    False
98    False
99    False
Length: 100, dtype: bool

In [39]:
joined_df['big_sale'] = big_sale_col
joined_df

,index,id_cart,total_cart,discountedTotal,userId,totalProducts,totalQuantity,id_product,title,price,quantity,total_product,discountPercentage,discountedPrice,big_sale
0,0,1,2328,1941,97,5,10,59,Spring and summershoes,20,3,60,8.71,55,False
1,0,1,2328,1941,97,5,10,88,TC Reusable Silicone Magic Washing Gloves,29,2,58,3.19,56,False
2,0,1,2328,1941,97,5,10,18,Oil Free Moisturizer 100ml,40,2,80,13.10,70,False
3,0,1,2328,1941,97,5,10,95,Wholesale cargo lashing Belt,930,1,930,17.67,766,True
4,0,1,2328,1941,97,5,10,39,Women Sweaters Wool,600,2,1200,17.20,994,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,19,20,315,279,75,5,8,66,Steel Analog Couple Watches,35,3,105,3.23,102,True
96,19,20,315,279,75,5,8,59,Spring and summershoes,20,1,20,8.71,18,False
97,19,20,315,279,75,5,8,29,Handcraft Chinese style,60,1,60,15.34,51,False
98,19,20,315,279,75,5,8,32,Sofa for Coffe Cafe,50,1,50,15.59,42,False


In [40]:
from datetime import datetime

In [41]:
now = datetime.now()

timestamp = datetime.timestamp(now)

joined_df['processed_ts'] = timestamp
joined_df

,index,id_cart,total_cart,discountedTotal,userId,totalProducts,totalQuantity,id_product,title,price,quantity,total_product,discountPercentage,discountedPrice,big_sale,processed_ts
0,0,1,2328,1941,97,5,10,59,Spring and summershoes,20,3,60,8.71,55,False,1.667988e+09
1,0,1,2328,1941,97,5,10,88,TC Reusable Silicone Magic Washing Gloves,29,2,58,3.19,56,False,1.667988e+09
2,0,1,2328,1941,97,5,10,18,Oil Free Moisturizer 100ml,40,2,80,13.10,70,False,1.667988e+09
3,0,1,2328,1941,97,5,10,95,Wholesale cargo lashing Belt,930,1,930,17.67,766,True,1.667988e+09
4,0,1,2328,1941,97,5,10,39,Women Sweaters Wool,600,2,1200,17.20,994,True,1.667988e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,19,20,315,279,75,5,8,66,Steel Analog Couple Watches,35,3,105,3.23,102,True,1.667988e+09
96,19,20,315,279,75,5,8,59,Spring and summershoes,20,1,20,8.71,18,False,1.667988e+09
97,19,20,315,279,75,5,8,29,Handcraft Chinese style,60,1,60,15.34,51,False,1.667988e+09
98,19,20,315,279,75,5,8,32,Sofa for Coffe Cafe,50,1,50,15.59,42,False,1.667988e+09


In [42]:
joined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               100 non-null    int64  
 1   id_cart             100 non-null    int64  
 2   total_cart          100 non-null    int64  
 3   discountedTotal     100 non-null    int64  
 4   userId              100 non-null    int64  
 5   totalProducts       100 non-null    int64  
 6   totalQuantity       100 non-null    int64  
 7   id_product          100 non-null    int64  
 8   title               100 non-null    object 
 9   price               100 non-null    int64  
 10  quantity            100 non-null    int64  
 11  total_product       100 non-null    int64  
 12  discountPercentage  100 non-null    float64
 13  discountedPrice     100 non-null    int64  
 14  big_sale            100 non-null    bool   
 15  processed_ts        100 non-null    float64
dtypes: bool(1

### 3.5 Dump the transformed data as CSV

In [43]:
joined_df.to_csv('processed.csv', index=False)